Run the following cell to verify that numba can use Cuda...

In [12]:
from numba import cuda

def bool_to_str(predicate: bool) -> str:
    if predicate:
        return 'yes'
    return 'no'

def is_cuda_available_str() -> str:
    return bool_to_str(cuda.is_available())

print(f'is cuda available? {is_cuda_available_str()}')

is cuda available? yes


Another simple solution to detect and print the devices from [numba documentation](https://numba.readthedocs.io/en/stable/cuda-reference/host.html#device-management):

In [13]:
from numba import cuda

if not cuda.detect():
    raise Exception("we do not have cuda :-(")

Found 1 CUDA devices
id 0    b'NVIDIA GeForce GTX 1650'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-052eb512-813b-9820-ab30-d99d3f99004a
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


Run the following cell for a short test of numba/cuda...

In [15]:
# This is a MAP example
from numba import cuda
import numpy as np 


@cuda.jit
def increment_by_one(an_array):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    if pos < an_array.size:  # Check array boundaries
        an_array[pos] += 1
        

if __name__ == "__main__":
    # build a big vector
    h_array = np.arange(1<<16)
    
    d_array = cuda.to_device(h_array)
    
    threads_per_block = 32*8 # 8 warps, 256 threads per block
    blocks_per_grid = (h_array.size + (threads_per_block - 1)) // threads_per_block
    increment_by_one[blocks_per_grid, threads_per_block](d_array)
    
    d_array.copy_to_host(h_array)
    
    for i in range(len(h_array)):
        assert h_array[i] == i+1, f'bad value at index {i}: {h_array[i]+1}'
    print('it woks!')

it woks!


From here, it is clear that some code are easy to do:
- binary transform
- gather
- scatter

That's what you did during the first week... Now let us try some simple PRAM algorithms. 

The main problem with PRAM model is that it is too simple to be applied directly to GPU. Indeed a GPU is not a simple vector processor but a set of vector processor. 

You already saw that a Cuda Grid contains some Cuda Blocks, and each Cuda Block contains some threads. 
The threads are organized into **warps**.

A warp is the logical Cuda view of a vector processor, with 32 elementary processors. Hence, threads are working in vector mode by group of 32... 

So, how to program PRAM algorithms in Cuda? Good question, thanks. 

Today we are working into a single block (we will see how to use different blocks concurrently later) with some warps. Remember that the maximum number of threads you can use into a block is limited to 1024. The warp is a group of 32 threads, so the maximum number of warps is 32!

Now, how to simulate PRAM into a block? For that purpose we need to synchronize the warps, to avoid race condition onto data. The simple solution to synchronizer all the threads into a block is to use the instruction `cuda.syncthreads()`. 

**Warning**: this instruction needs to be used by **all** the threads of the block. It is like a barrier, and it opens when all the threads reach it only...

To avoid high latency and synchronisation problem, notice that all the data should be first loaded into block shared memory... This kind of memory should be allocated using the `numba.cuda.array(shape, dtype)` method.

## First exercise
The objective here is to write a Cuda algorithm that calculates the maximum of 32 values into a single block. Your implementation should be added at line 21!

32 threads means a single warp, so no synchronisation is needed here...

In [4]:
from __future__ import annotations

import numpy as np
from numba import cuda, core
from numba.np.numpy_support import from_dtype


class CudaMaximum:
    _kernels_cache = {}

    def __init__(self: CudaMaximum) -> None:
        pass

    @staticmethod
    def _gpu_kernel_factory(np_type):
        """Factory of kernels for the maximum problem...

        This function returns a Cuda Kernel that does the maximum of some data using a single block."""

        def kernel(d_input, d_maximum) -> None:
            tid = cuda.threadIdx.x
            shared = cuda.shared.array(shape=32, dtype=d_input.dtype)
            # do it here
            shared[tid] = d_input[tid]
            cuda.syncthreads()
            
            j = 1
            cuda.syncthreads()
            while j < 32:
                if tid + j <= (32-1):
                    shared[tid] = max(shared[tid], shared[tid+j])
                cuda.syncthreads()
                j = j * 2
            if tid == 0:
                d_maximum[0] = shared[tid]
            
        return cuda.jit(kernel)

    @staticmethod
    def _compile(dtype):

        key = dtype
        if key not in CudaMaximum._kernels_cache:
            CudaMaximum._kernels_cache[key] = CudaMaximum._gpu_kernel_factory(from_dtype(dtype))

        return CudaMaximum._kernels_cache[key]

    def __call__(self, buffer, maximum, stream=cuda.default_stream()):
        """Computes the maximum.

        :param buffer: A device array, containing the data.
        :param stream: Optional CUDA stream in which to perform the calculations.
                    If no stream is specified, the default stream of 0 is used.
        :return: ``None``
        """

        # ensure 1d array
        if buffer.ndim != 1:
            raise TypeError("only support 1D array")

        # ensure size > 0
        if buffer.size < 1:
            raise ValueError("array's length is 0")

        # ensure size == 1
        if maximum.size != 1:
            raise ValueError("array's length is not zero")

        # ensure size <= 32
        if buffer.size > 32:
            raise ValueError("array's length is too big")

        kernel = self._compile(buffer.dtype)

        # Perform the maximum on the GPU
        nb_threads = buffer.size
        nb_blocks = 1

        start_event = cuda.event(True)
        stop_event = cuda.event(True)

        start_event.record(stream=stream)
        kernel[nb_blocks, nb_threads, stream](buffer, maximum)
        stop_event.record(stream=stream)
        stop_event.synchronize()
        ct = cuda.event_elapsed_time(start_event, stop_event)
        print(f"kernel computation time is {ct} ms")


if __name__ == "__main__":
    def check(maximum, array, msg):
        error = 0
        for x in array:
            if x > maximum:
                error = error + 1
        if error > 0:
            print(f"{msg} does not work: {error} errors generated")
        else:
            print(f"{msg} seems to work")


    def test(h_buffer):
        d_buffer = cuda.to_device(h_buffer)
        d_maximum = cuda.device_array(shape=1, dtype=d_buffer.dtype)

        maxer = CudaMaximum()
        maxer(d_buffer, d_maximum)

        h_maximum = d_maximum.copy_to_host()

        check(h_maximum[0], h_buffer, "maximum")


    core.config.CUDA_LOW_OCCUPANCY_WARNINGS = False
    buffer = np.random.randint(low=0, high=1 << 20, size=32, dtype=np.int32)
    test(buffer)


kernel computation time is 110.48009490966797 ms
maximum seems to work


## Second exercise
The objective here is to write a Cuda algorithm that calculates the maximum of 1024 values into a single block. Your implementation should be added at line 21!

Take care: use threads' synchronization to simulate the PRAM algorithm with multiple warps!

In [8]:
from __future__ import annotations

import numpy as np
from numba import cuda, core
from numba.np.numpy_support import from_dtype


class CudaMaximum:
    _kernels_cache = {}

    def __init__(self: CudaMaximum) -> None:
        pass

    @staticmethod
    def _gpu_kernel_factory(np_type, nb_threads):
        """Factory of kernels for the maximum problem...

        This function returns a Cuda Kernel that does the maximum of some data using a single block."""

        def kernel(d_input, d_maximum) -> None:
            tid = cuda.threadIdx.x
            shared = cuda.shared.array(shape=nb_threads, dtype=d_input.dtype)
            # do it here
            shared[tid] = d_input[tid]
            cuda.syncthreads()
            
            j = 1
            while j < nb_threads:
                if tid + j < nb_threads:
                    temp = shared[tid + j]
                cuda.syncthreads()
                if tid + j < nb_threads:
                    temp = shared[tid+j]
                    if shared[tid] < temp:
                        shared[tid] = temp
                cuda.syncthreads()
                j= j*2
            
            if tid ==0:
                d_maximum[0] = shared[0]
        return cuda.jit(kernel)

    @staticmethod
    def _compile(dtype, nb_threads):

        key = dtype, nb_threads
        if key not in CudaMaximum._kernels_cache:
            CudaMaximum._kernels_cache[key] = CudaMaximum._gpu_kernel_factory(from_dtype(dtype), nb_threads)

        return CudaMaximum._kernels_cache[key]

    def __call__(self, buffer, maximum, stream=cuda.default_stream()):
        """Computes the maximum.

        :param buffer: A device array, containing the data.
        :param stream: Optional CUDA stream in which to perform the calculations.
                    If no stream is specified, the default stream of 0 is used.
        :return: ``None``
        """

        # ensure 1d array
        if buffer.ndim != 1:
            raise TypeError("only support 1D array")

        # ensure size > 0
        if buffer.size < 1:
            raise ValueError("array's length is 0")

        # ensure size == 1
        if maximum.size != 1:
            raise ValueError("array's length is not zero")

        # ensure size < 1024+1
        if buffer.size > 1024:
            raise ValueError("array's length is too big")

        # Perform the maximum on the GPU
        nb_threads = 1024
        nb_blocks = 1

        kernel = self._compile(buffer.dtype, nb_threads)

        start_event = cuda.event(True)
        stop_event = cuda.event(True)
        start_event.record(stream=stream)
        kernel[nb_blocks, nb_threads, stream](buffer, maximum)
        stop_event.record(stream=stream)
        stop_event.synchronize()
        ct = cuda.event_elapsed_time(start_event, stop_event)
        print(f"kernel computation time is {ct} ms")


if __name__ == "__main__":
    def check(maximum, array, msg):
        error = 0
        for x in array:
            if x > maximum:
                error = error + 1
        if error > 0:
            print(f"{msg} does not work: {error} errors generated")
        else:
            print(f"{msg} seems to work")


    def test(h_buffer):
        d_buffer = cuda.to_device(h_buffer)
        d_maximum = cuda.device_array(shape=1, dtype=d_buffer.dtype)

        maxer = CudaMaximum()
        maxer(d_buffer, d_maximum)

        h_maximum = d_maximum.copy_to_host()

        check(h_maximum[0], h_buffer, "maximum")


    core.config.CUDA_LOW_OCCUPANCY_WARNINGS = False
    test(np.random.randint(low=0, high=1 << 20, size=1024, dtype=np.int32))


kernel computation time is 211.63623046875 ms
maximum seems to work


## Third exercise
You may have notice that the first two exercises works with 1024 values only. 
If you remove the line:
```python
    core.config.CUDA_LOW_OCCUPANCY_WARNINGS = False
```
Then you will see a warning from Cuda saying that the GPU is under-utilized.
Indeed, we ran a single block, onto a single SMP (*Streaming Multi-Processor*), while there is plenty of SMP (from 4 to tens)...
To overcome this problem, the solution is quite simple but not obvious.

First, you have to launch multiple block, but no more than 256 to avoid registers' pressure (shared memory is simulated using registers, and so too big shared memory implied low number of registers per thread, and so reduced efficiency...). 

Then, you obtain one maximum value per block, and this value should be saved into a specific isolated memory location: so you need an array of maximums of size equals to the number of blocks...

At last, well you have something like 256 maximum values... Hum, it is where the first exercise is useful ;-)

Modify you implementation to work with many values (something like 1 millions, at least)...

In [9]:
from __future__ import annotations

import numpy as np
from numba import cuda, core
from numba.np.numpy_support import from_dtype


class CudaMaximum:
    _kernels_1_cache = {}
    _kernels_2_cache = {}

    def __init__(self: CudaMaximum) -> None:
        pass

    @staticmethod
    def _gpu_kernel_factory_1(np_type, nb_threads):
        """Factory of kernels for the maximum problem...

        This function returns a Cuda Kernel that does the maximum of some data using a multiple block."""

        def kernel(d_input, d_maximum) -> None:
            gtid = cuda.grid(1)
            ltid = cuda.threadIdx.x
            bdim = cuda.blockDim.x
            
            shared = cuda.shared.array(shape=nb_threads, dtype=d_input.dtype)
            if gtid < d_input.size:
                shared[ltid] = d_input[gtid]
            else: 
                shared[ltid]= 0
            cuda.syncthreads
            
            j = 1
            while j < cuda.blockDim.x:
                if(gtid + j < d_input.size) and (ltid + j < bdim):
                    temp = shared[ltid + j]
                cuda.syncthreads()
                if(gtid + j < d_input.size) and (ltid + j < bdim):
                    if shared[ltid] < temp:
                        shared[ltid] = temp
                cuda.syncthreads()
                j = j * 2
            if ltid == 0:
                d_maximum[cuda.blockIdx.x] = shared[0]
            
            
        return cuda.jit(kernel)

    @staticmethod
    def _compile_step1(dtype, nb_threads):

        key = dtype, nb_threads
        if key not in CudaMaximum._kernels_1_cache:
            CudaMaximum._kernels_1_cache[key] = CudaMaximum._gpu_kernel_factory_1(from_dtype(dtype), nb_threads)

        return CudaMaximum._kernels_1_cache[key]

    @staticmethod
    def _gpu_kernel_factory_2(np_type, nb_threads):
        """Factory of kernels for the maximum problem...

        This function returns a Cuda Kernel that does the maximum of some data using a single block."""

        def kernel(d_input, d_maximum) -> None:
            tid = cuda.threadIdx.x
            shared = cuda.shared.array(shape=nb_threads, dtype=d_input.dtype)
            # do it here
            
            
            
            

        return cuda.jit(kernel)

    @staticmethod
    def _compile_step2(dtype, nb_threads):

        key = dtype
        if key not in CudaMaximum._kernels_2_cache:
            CudaMaximum._kernels_2_cache[key] = CudaMaximum._gpu_kernel_factory_2(from_dtype(dtype), nb_threads)

        return CudaMaximum._kernels_2_cache[key]

    def __call__(self, buffer, maximum, stream=0):
        """Computes the maximum.

        :param buffer: A device array, containing the data.
        :param stream: Optional CUDA stream in which to perform the calculations.
                       If no stream is specified, the default stream of 0 is used.
        :return: ``None``
        """

        # ensure 1d array
        if buffer.ndim != 1:
            raise TypeError("only support 1D array")

        # ensure size > 0
        if buffer.size < 1:
            raise ValueError("array's length is 0")

        # ensure size == 1
        if maximum.size != 1:
            raise ValueError("array's length is not zero")

        # Perform the maximum per block on the GPU
        nb_threads = 256
        kernel_1 = self._compile_step1(buffer.dtype, nb_threads)

        start_event = cuda.event(True)
        stop_event = cuda.event(True)
        start_event.record(stream=stream)

        while buffer.size > 256:
            nb_blocks = (buffer.size + nb_threads - 1) // nb_threads
            print(f"launch {nb_blocks} for {nb_threads * nb_blocks} threads")

            temp = cuda.device_array(shape=nb_blocks, dtype=buffer.dtype)

            kernel_1[nb_blocks, nb_threads, stream](buffer, temp)

            cuda.synchronize()

            buffer = temp

        # second step...
        kernel_2 = self._compile_step2(buffer.dtype, buffer.size)
        kernel_2[1, buffer.size, stream](buffer, maximum)

        stop_event.record(stream=stream)
        stop_event.synchronize()
        ct = cuda.event_elapsed_time(start_event, stop_event)
        print(f"kernel computation time is {ct} ms")



if __name__ == "__main__":
    def check(maximum, array, msg):
        error = 0
        for x in array:
            if x > maximum:
                error = error + 1
        if error > 0:
            print(f"{msg} does not work: {error} errors generated")
        else:
            print(f"{msg} seems to work")


    def test(h_buffer):
        d_buffer = cuda.to_device(h_buffer)
        d_maximum = cuda.device_array(shape=1, dtype=d_buffer.dtype)

        maxer = CudaMaximum()
        maxer(d_buffer, d_maximum)

        h_maximum = d_maximum.copy_to_host()
        print(f"Maximum is {h_maximum[0]}")

        check(h_maximum[0], h_buffer, "maximum")


    core.config.CUDA_LOW_OCCUPANCY_WARNINGS = False
    buffer = np.random.randint(low=0, high=1 << 20, size=1 << 28, dtype=np.int32)
    test(buffer)


launch 1048576 for 268435456 threads
launch 4096 for 1048576 threads
launch 16 for 4096 threads
kernel computation time is 282.4485168457031 ms
Maximum is 915371
maximum does not work: 34093981 errors generated
